# Exercise 3: MAP Classifier

In this assignment you will implement Baysian learning

## Read the following instructions carefully:

1. This jupyter notebook contains all the step by step instructions needed for this part of the exercise.
2. Write vectorized code whenever possible.
3. You are responsible for the correctness of your code and should add as many tests as you see fit. Tests will not be graded nor checked.
4. Write your functions in this notebook only.
5. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/) and [numpy](https://www.numpy.org/devdocs/reference/) only. 
6. Your code must run without errors. During the environment setup, you were given a specific version of `numpy` to install. Changes of the configuration we provided are at your own risk. Code that cannot run will also earn you the grade of 0.
7. Write your own code. Cheating will not be tolerated. 
8. Submission includes this notebook and the answers to the theoretical part. Answers to qualitative questions should be written in markdown cells (with $\LaTeX$ support).
9. You can add additional functions.
10. Submission: zip only the completed jupyter notebook and the PDF with your solution for the theory part. Do not include the data or any directories. Name the file `ID1_ID2.zip` and submit **only one copy of the assignment**.

## In this exercise you will perform the following:
1. Uderstand Conditional Independence concept 
1. Implement density estimation using MLE
1. Implement a Naive Bayes Classifier based on Uni-Normal distribution
1. Implement a Full Bayes Classifier based on Multi-Normal distribution
1. Implement a Discrete Naive Bayes Classifier

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st 
import math

plt.style.use('seaborn')
sns.set(style="darkgrid")
sns.set(font_scale=1.2)

# Conditional independence  
Define 3 random variables (RV) $X, Y, C$ s.t.:  
1. $X, Y$ and $C$ are all binary
2. The following conditions hold:  
    * P(X=0) = 0.3
    * P(Y=0) = 0.3
    * P(C=0) = 0.5
3. $X$ and $Y$ are not independent
4. $X$ and $Y$ are conditionaly independent given $C$ $(X \perp\!\!\!\perp Y |C)$

In order to define those RV you need to fill the distributions (represent as python dictionaries) below
and then write a function that prove that conditions 3 (`is_X_Y_depndendent`) and 4 (`is_X_Y_given_C_independent`) holds.

Rcall that:   
1. $P(X|Y) = \frac{P(X, Y)}{P(Y)}$  
2. $P(X, Y|C) = \frac{P(X, Y, C)}{P(C)}$
3. $(X \perp\!\!\!\perp Y |C)$   iff  
$\forall x, y,c$: $p(X=x,Y=y|C=c)=p(X=x|C=c)p(Y=y|C=c)$  

Make sure that all the probabilities are valid! 

In [2]:
# You need to fill the None value with valid probabilities
X = {0: 0.3, 1: 0.7}  # P(X=x)
Y = {0: 0.3, 1: 0.7}  # P(Y=y)
C = {0: 0.5, 1: 0.5}  # P(C=c)

# TODO: eyal - I think that for X, Y to be not independent we simply need to provide one value (for example: (0, 0)) to NOT be the multiplication of p(x=0) times p(y=0). 
                # And of course probs vector needs to sum to 1.
X_Y = {
    (0, 0): 0.25,
    (0, 1): 0.25,
    (1, 0): 0.25,
    (1, 1): 0.25
}  # P(X=x, Y=y)

X_C = {
    (0, 0): 0.25,
    (0, 1): 0.25,
    (1, 0): 0.25,
    (1, 1): 0.25
}  # P(X=x, C=y)

Y_C = {
    (0, 0): 0.25,
    (0, 1): 0.25,
    (1, 0): 0.25,
    (1, 1): 0.25
}  # P(Y=y, C=c)

X_Y_C = {
    (0, 0, 0): 0.125, # P(X=x, Y=y, C=c) = P(X=x, Y=y | C=c) * p(C=c) = P(X=x | C=c) * P(Y=y | C=c) * p(C=c) = P(X=0 | C=0) * P(Y=0 | C=0) * p(C=0) = 
    # 0.5 * 0.5 * 0.5 = 0.125
    # explanation for the above calculation: P(A, B) = P(A | B) * P(B) (because of the fact that P(A | B) = P(A, B) / P(B))
    # TODO: not sure if we need to write this down or not.
    (0, 0, 1): 0.125,
    (0, 1, 0): 0.125,
    (0, 1, 1): 0.125,
    (1, 0, 0): 0.125,
    (1, 0, 1): 0.125,
    (1, 1, 0): 0.125,
    (1, 1, 1): 0.125
}  # P(X=x, Y=y, C=c)

In [3]:
# You may assume that X, Y and C have the same support (e.g. they are defined on the same space)
# Note: since python suffer from numerical instability you may want to use np.isclose instead of the `==` operator
def is_X_Y_depndendent(X, Y, X_Y):
    """
    return True iff X and Y are depndendent
    """
    # let's calculate for each values, the probabilty that it'd have if X & Y are indepndendent
    for X_Y_values in X_Y:
        x_val, y_val = X_Y_values
        prob_x, prob_y = X[x_val], Y[y_val]
        prob_if_indepndendent = prob_x * prob_y
        real_prob = X_Y[X_Y_values]

        if not np.isclose(prob_if_indepndendent, real_prob):
            # it's enough that just one of the values shows this to be depndendent
            print(f"X and Y are depndendent! \nThe probability for X={x_val} & Y={y_val} if they were independent is: {prob_if_indepndendent}, but it's: {real_prob} \n")
            return True
        
    print("It seems that X and Y are independent!")
    return False


def is_X_Y_given_C_independent(X, Y, C, X_C, Y_C, X_Y_C):
    """
    return True iff X_given_C and Y_given_C are indepndendent
    """
    # let's calculate for each values, the probabilty that it'd have if X & Y are depndendent given c
    for X_Y_C_values in X_Y_C:
        x_val, y_val, c_val = X_Y_C_values
        prob_x, prob_y, prob_c = X[x_val], Y[y_val], C[c_val]

        p_x_given_c = X_C[(x_val, c_val)] / prob_c
        p_y_given_c = Y_C[(y_val, c_val)] / prob_c

        prob_if_indepndendent = p_x_given_c * p_y_given_c * prob_c
        real_prob = X_Y_C[X_Y_C_values]

        if not np.isclose(prob_if_indepndendent, real_prob):
            # it's enough that just one of the values shows this to be depndendent
            print(f"X and Y are depndendent given C! \nThe probability for X={x_val} & Y={y_val} given C={c_val} if they were independent is: {prob_if_indepndendent}, but it's: {real_prob} \n")
            return False
    
    print("It seems that X & Y are independent given C!")
    # we are supposed to get here in our example
    return True


In [4]:
assert is_X_Y_depndendent(X, Y, X_Y)
assert is_X_Y_given_C_independent(X, Y, C, X_C, Y_C, X_Y_C)

X and Y are depndendent! 
The probability for X=0 & Y=0 if they were independent is: 0.09, but it's: 0.25 

It seems that X & Y are independent given C!


# Maximum Likelihood estimation  

In probability theory and statistics, the Poisson distribution is a discrete probability distribution that expresses the probability of a given number of events occurring in a fixed interval of time or space if these events occur with a known constant mean rate and independently of the time since the last event.  
The paramter of this distribution is the rate of events in that given time interval, annotated $\lambda$  
if $X$~$Pois(\lambda)$  
then $p(X=k|\lambda) = \frac{\lambda^ke^{-\lambda}}{k!}$  
Where $X$ is a RV $\lambda$ is the rate and $p$ is the pmf

Implement the function `poisson_log_pmf`

In [5]:
def poisson_log_pmf(k, rate):
    """
    k: A discrete instance
    rate: poisson rate parameter (lambda)

    return the log pmf value for instance k given the rate
    """
    nominator = rate ** k * np.e ** -rate
    denominator = math.factorial(k)
    return np.log(nominator / denominator)

In the file poisson_1000_samples.csv there are 1000 points drawn from some poisson distribution with a fixed paramter $\lambda$  

In the following section you are going to find a rate that maximizes the likelihood function. You will do this in 2 diffrent ways:
1. Iterative (`possion_iterative_mle`): given a list of possible rates (`rates`), calculate the log likelihood value for each rate and return the rate that has the maximum value
2. Analytic (`possion_analytic_mle`): read the following blog: https://www.statology.org/mle-poisson-distribution/. This blog demonstrate how to derive the MLE of a poisson distribution. Understande the process and implement the function.


In [6]:
poisson_samples = pd.read_csv('data/poisson_1000_samples.csv').values.flatten()
rates = np.linspace(1e-20, 20, num=1000)

In [7]:
def get_poisson_log_likelihoods(samples, rates):
    """
    samples: set of univariate discrete observations
    rates: an iterable of rates to calculate log-likelihood by.

    return: 1d numpy array, where each value represent that log-likelihood value of rates[i]
    """
    # TODO: this doesn't work well (the break shouldn't be there)
    log_likelihoods_per_rate = list()
    for rate in rates:
        samples_log_likelihoods = [poisson_log_pmf(k=k, rate=rate) for k in samples]

        # because it's a log likelihoopd, the multiplication can be presented as summation: 
        # log( p(D | lambda) ) = log( p(x1 | lambda) * p(x2 | lambda) ... ) = log( p(x1 | lambda) ) + log( p(x2 | lambda) ) them.
        log_likelihood = 0.0
        for sample_log_likelihood in samples_log_likelihoods:
            log_likelihood = log_likelihood + sample_log_likelihood

        log_likelihoods_per_rate.append(log_likelihood)

    return log_likelihoods_per_rate

    
def possion_iterative_mle(samples, rates):
    """
    samples: set of univariate discrete observations
    rate: a rate to calculate log-likelihood by.

    return: the rate that maximizes the likelihood 
    """
    rate = 0.
    log_likelihoods = get_poisson_log_likelihoods(samples, rates)
    # Your code goes here
    max_log_likelihood = np.max(log_likelihoods)
    rate_index = log_likelihoods.index(max_log_likelihood)
    rate = rates[rate_index]

    # End of your code
    return rate

def possion_analytic_mle(samples):
    """
    samples: set of univariate discrete observations

    return: the rate that maximizes the likelihood
    """
    # the poisson analytical MLE, as described in the blog, is simply the mean of the samples:
    samples_mean = np.mean(samples)
    return samples_mean


In [ ]:
x = rates
y = get_poisson_log_likelihoods(poisson_samples, rates)

iterative_rate = possion_iterative_mle(poisson_samples, rates)
analytic_rate = possion_analytic_mle(poisson_samples)

plt.plot(x, y)
plt.axvline(x=iterative_rate, linestyle=':', c='r', label=f"iterative: {iterative_rate:.2f}")
plt.axvline(x=analytic_rate, linestyle='--', c='g', label=f"analytic: {analytic_rate:.2f}")
plt.legend()
plt.show()


# Normal Naive Bayes Classifier Vs Normal Full Bayes Classifier
In the following section we are going to compare 2 models on a given dataset. <br>
The 2 classifiers we are going to test are:
1. Naive Bayes classifer.<br>
1. Full Bayes classifier.<br>
Recall that a Naive Bayes classifier makes the following assumption :<br> 
## $$ p(x_1, x_2, ..., x_n|A_j) = \Pi p(x_i | A_j) $$
But the full Bayes classifier will not make this assumption.<br>

### The Data Story

In a faraway land called **Randomistan** there is a rare animal called the **Randomammal**.<br> 
We have gathered data about this unique animal to help the **randomian** researchers in observing this beast. <br>
For a 1000 days straight we have measured the temperature and the humidity in Randomistan and whether the Randomammal was spotted or not. <br>
The well known randomian **Bob** is a bit of a lazy researcher so he likes to keep things simple, and so he assumes that the temperature and the humidity are independent given the class. <br>
**Alice** on the other hand is a hard working researcher and does not make any assumptions, she's young and is trying to gain some fame in the randomian community.

The dataset contains 2 features (**Temperature**, **Humidity**) alongside a binary label (**Spotted**) for each instance.<br>

We are going to test 2 different classifiers :
* Naive Bayes Classifier (Bob)
* Full Bayes Classifier. (Alice)

Both of our researchers assume that our features are normally distributed. But while Bob with his Naive classifier will assume that the features are independent, Alice and her Full Bayes classifier will not make this assumption.<br><br>
Let's start off by loading the data (train, test) into a pandas dataframe and then converting them
into numpy arrays.<br>
The datafiles are :
- randomammal_train.csv
- randomammal_test.csv

In [ ]:
# Load the train and test set into a pandas dataframe and convert them into a numpy array.
# The columns order: ['Temp', 'Humidity', 'Spotted']
train_set = pd.read_csv('data/randomammal_train.csv').values
test_set = pd.read_csv('data/randomammal_test.csv').values

# Data Visualization
Draw a scatter plot of the training data where __x__=Temerature and **y**=Humidity. <br>
Use color to distinguish points from different classes.<br>
Stop for a minute to think about Alice and Bob's approaches and which one you expect to work better.

In [ ]:
plt.style.use('seaborn')
sns.set(style="darkgrid")
sns.set(font_scale=1.2)

# Your code goes here.
plt.figure(figsize=(10, 6))
mask_not_spotted = train_set[:, -1] == 0
mask_spotted = train_set[:, -1] == 1

plt.scatter(x=train_set[mask_not_spotted][:, 0], y=train_set[mask_not_spotted][:, 1], marker='o', color='blue', alpha=0.3)
plt.scatter(x=train_set[mask_spotted][:, 0], y=train_set[mask_spotted][:, 1], marker='o', color='red', alpha=0.3)

plt.title('Scatter Plot Humidity & Temperature'.title())
plt.xlabel('Temperature')
plt.ylabel('Humidity')
plt.legend(['Not Spotted', 'Spotted'], loc = 'upper right')

plt.show()

## Bob's Naive Model

Start with implementing the [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) probability density function in the next cell: 
$$ \frac{1}{\sqrt{2\pi \sigma^2}} \cdot e ^{-\frac{(x - \mu)^2}{2\sigma^2}} $$
Where :
* $\mu$ is the distribution mean.
* $\sigma$ is the distribution standard deviation.

In [ ]:
def normal_pdf(x, mean, std):
    """
    Calculate normal desnity function for a given x, mean and standrad deviation.
 
    Input:
    - x: A value we want to compute the distribution for.
    - mean: The mean value of the distribution.
    - std:  The standard deviation of the distribution.
 
    Returns the normal distribution pdf according to the given mean and std for the given x.    
    """
    nomimnator = np.exp( -( (x-mean)**2 ) / ( 2 * std **2 ) )
    denomimnator = np.sqrt( 2 * np.pi * std**2 )
    return nomimnator / denomimnator



Implement the **NaiveNormalClassDistribution** in the next cell and build a distribution object for each class.
Recall that when using the naive assumption, we assume our features are indepenent given the class. Meaning:
$$ P(x_1, x_2 | Y) = p(x_1 | Y) \cdot p(x_2 | Y)$$


Since we assume our features are normally distributed we need to find the mean and std for each feature in order for us to compute those probabilites. 

In [ ]:
class NaiveNormalClassDistribution():
    def __init__(self, dataset, class_value):
        """
        A class which encapsulates the relevant parameters(mean, std) for a class conditinoal normal distribution.
        The mean and std are computed from a given data set.
        
        Input
        - dataset: The dataset as a 2d numpy array, assuming the class label is the last column
        - class_value : The class to calculate the parameters for.
        """
        self.class_value_mask = np.where(dataset[: , -1] == class_value)
        self.means = np.mean(dataset[self.class_value_mask][: , :-1], axis=0)
        self.stds = np.std(dataset[self.class_value_mask][: , :-1], axis=0)
        self.dataset = dataset
        self.class_value = class_value

    def get_prior(self):
        """
        Returns the prior porbability of the class according to the dataset distribution.
        """
        return np.sum(self.dataset[: , -1] == self.class_value) / len(self.dataset)
    
    def get_instance_likelihood(self, x):
        """
        Returns the likelihhod porbability of the instance under the class according to the dataset distribution.
        """
        xs_likelihoods = [normal_pdf(x_i, mean, std) for x_i, mean, std in zip(x, self.means, self.stds)]

        # the instance likelihood is the multiplication of the likelihood of all the features (thanks to the NAIVENESS)        
        return np.prod(xs_likelihoods)

    def get_instance_posterior(self, x):
        """
        Returns the posterior porbability of the instance under the class according to the dataset distribution.
        * Ignoring p(x)
        """
        instance_likelihood = self.get_instance_likelihood(x)
        prior = self.get_prior()
        return instance_likelihood * prior

In [ ]:
# Build the a NaiveNormalClassDistribution for each class.
naive_normal_CD_0 = NaiveNormalClassDistribution(train_set, 0)
naive_normal_CD_1 = NaiveNormalClassDistribution(train_set, 1)

Implement the **MAPClassifier** class and build a MAPClassifier object containing the 2 distribution objects you just made above.

In [ ]:
class MAPClassifier():
    def __init__(self, ccd0 , ccd1):
        """
        A Maximum a posteriori classifier. 
        This class will hold 2 class distributions, one for class 0 and one for class 1, and will predict an instance
        by the class that outputs the highest posterior probability for the given instance.
    
        Input
            - ccd0 : An object contating the relevant parameters and methods for the distribution of class 0.
            - ccd1 : An object contating the relevant parameters and methods for the distribution of class 1.
        """
        self.dist0 = ccd0
        self.dist1 = ccd1
    
    def predict(self, x):
        """
        Predicts the instance class using the 2 distribution objects given in the object constructor.
    
        Input
            - An instance to predict.
        Output
            - 0 if the posterior probability of class 0 is higher and 1 otherwise.
        """
        if self.dist0.get_instance_posterior(x) >= self.dist1.get_instance_posterior(x):
            return 0
        
        # if got here, then class 1 has higher posterior
        return 1

In [ ]:
naive_normal_classifier = MAPClassifier(naive_normal_CD_0, naive_normal_CD_1)

### Evaluate model
Implement the **compute_accuracy** function in the next cell. Use it and the 2 distribution objects you created to compute the accuracy on the test set.

In [ ]:
def compute_accuracy(testset, map_classifier):
    """
    Compute the accuracy of a given a testset using a MAP classifier object.
    
    Input
        - testset: The testset for which to compute the accuracy (Numpy array). where the class label is the last column
        - map_classifier : A MAPClassifier object capable of prediciting the class for each instance in the testset.
        
    Ouput
        - Accuracy = #Correctly Classified / #testset size
    """
    preds = [map_classifier.predict(row) for row in testset[: , :-1]]
    return np.sum(preds == testset[: , -1]) / len(testset)

In [ ]:
# Compute the naive model accuracy and store it in the naive accuracy variable.
naive_accuracy = compute_accuracy(test_set, naive_normal_classifier)
naive_accuracy

## Alice's Full Model

Start with Implementing the [multivariate normal](https://en.wikipedia.org/wiki/Multivariate_normal_distribution) distribution probability density function in the next cell.

## $$ (2\pi)^{-\frac{d}{2}} det(\Sigma )^{-\frac{1}{2}} \cdot e ^{-\frac{1}{2}(x-\mu)^T \Sigma ^ {-1} (x - \mu) }$$

Where : 
* $\mu$ is the distribution mean vector. (length 2 in our case)
* $\Sigma$ Is the distribution covarince matrix. (size 2x2 in our case)

In [ ]:

xs = np.ones(3)
mean_vec = [0, 0, 0]
cov_i_matrix = np.eye(3)

y = st.multivariate_normal.pdf(xs, mean_vec, cov_i_matrix)

In [ ]:
def multi_normal_pdf(x, mean, cov):
    """
    Calculate multi variable normal desnity function for a given x, mean and covarince matrix.
 
    Input:
    - x: A value we want to compute the distribution for.
    - mean: The mean vector of the distribution.
    - cov:  The covariance matrix of the distribution.
 
    Returns the normal distribution pdf according to the given mean and var for the given x.    
    """
    return ( 2 * np.pi) ** ( -len(x) / 2)  *  ( np.linalg.det(cov) ** -0.5 )  *  np.exp( -0.5 * (x-mean).T @ np.linalg.inv(cov) @ (x-mean))

In [ ]:
pdf_y = multi_normal_pdf(xs, mean_vec, cov_i_matrix)
assert np.isclose(y, pdf_y)

Implement the **MultiNormalClassDistribution** and build a distribution object for each class.

In the full bayes model we will not make any simplyfing assumptions, meaning, we will use a multivariate normal distribution. <br>
And so, we'll need to compute the mean of each feature and to compute the covariance between the features to build the covariance matrix.


In [ ]:
class MultiNormalClassDistribution():
    
    
    def __init__(self, dataset, class_value):
        """
        A class which encapsulate the relevant parameters(mean, cov matrix) for a class conditinoal multi normal distribution.
        The mean and cov matrix (You can use np.cov for this!) will be computed from a given data set.
        
        Input
        - dataset: The dataset as a numpy array
        - class_value : The class to calculate the parameters for.
        """
        self.class_value_mask = np.where(dataset[: , -1] == class_value)
        self.means = np.mean(dataset[self.class_value_mask][: , :-1], axis=0)
        self.cov = np.cov(dataset[self.class_value_mask][: , :-1].T)
        self.dataset = dataset
        self.class_value = class_value

    def get_prior(self):
        """
        Returns the prior porbability of the class according to the dataset distribution.
        """
        return np.sum(self.dataset[: , -1] == self.class_value) / len(self.dataset)
    
    def get_instance_likelihood(self, x):
        """
        Returns the likelihood of the instance under the class according to the dataset distribution.
        """
        return multi_normal_pdf(x, self.means, self.cov)
    
    def get_instance_posterior(self, x):
        """
        Returns the posterior porbability of the instance under the class according to the dataset distribution.
        * Ignoring p(x)
        """
        instance_likelihood = self.get_instance_likelihood(x)
        prior = self.get_prior()
        return instance_likelihood * prior

In [ ]:
# Build the a MultiNormalClassDistribution for each class.
multi_normal_CD_0 = MultiNormalClassDistribution(train_set, 0)
multi_normal_CD_1 = MultiNormalClassDistribution(train_set, 1)

build a MAPClassifier object contating the 2 distribution objects you just made above.

In [ ]:
multi_normal_classifier = MAPClassifier(multi_normal_CD_0, multi_normal_CD_1)

### Evaluate model
Use the **compute_accuracy** function and the 2 distribution objects you created to compute the accuracy on the test set.

In [ ]:
# Compute the naive model accuracy and store it in the naive accuracy variable.
full_accuracy = compute_accuracy(test_set, multi_normal_classifier)
full_accuracy

## Results

Use a plot bar to showcase the models accuracy.

In [ ]:
# Bar plot of accuracy of each model side by side.
plt.figure(figsize=(12, 8))
plt.bar(x=['Naive', 'Full'], height=[naive_accuracy, full_accuracy])
plt.title("Naive vs Full accuracy comparison", fontdict={"fontsize": 20})
plt.ylabel("Accuracy")

# Comparing Max a posteriori, prior, and likelihood results

For each of the classifiers above (naive Bayes and full Bayes, in which we compare posterior probabilities), we explore how classifiers would perform if we compare (1) only prior probabilities or (2) only likelihoods. 

In this section, you will implement MaxPrior and MaxLikelihood classifiers similarly to MAPClassifier, and then visualize the performance of the three models (MAP, MaxPrior, and MaxLikelihood) for each of the examples of above (naive Bayes and full Bayes).

For example, your visualization can be a graph where accuracy is the y-axis, "MaxPrior", "MaxLikelihood", and "MAP" are the x-axis values, and at each x-value, there will be two bars - one for the naive Bayes, and one for the full Bayes.  

Other graphs (that make sense / are intuitive) will be accepted as well.

Implement the **MaxPrior** class and build a MaxPrior object like you did above with the **MAPClassifier**.

In [ ]:
class MaxPrior():
    def __init__(self, ccd0 , ccd1):
        """
        A Maximum prior classifier. 
        This class will hold 2 class distributions, one for class 0 and one for class 1, and will predicit an instance
        by the class that outputs the highest prior probability for the given instance.
    
        Input
            - ccd0 : An object contating the relevant parameters and methods for the distribution of class 0.
            - ccd1 : An object contating the relevant parameters and methods for the distribution of class 1.
        """
        self.dist0 = ccd0
        self.dist1 = ccd1
    
    def predict(self, x):
        """
        Predicts the instance class using the 2 distribution objects given in the object constructor.
    
        Input
            - An instance to predict.
        Output
            - 0 if the posterior probability of class 0 is higher and 1 otherwise.
        """
        if self.dist0.get_prior() >= self.dist1.get_prior():
            return 0
            
        return 1


Implement the **MaxLikelihood** class and build a MaxLikelihood object like you did above with the **MAPClassifier**.

In [ ]:
class MaxLikelihood():
    def __init__(self, ccd0 , ccd1):
        """
        A Maximum Likelihood classifier. 
        This class will hold 2 class distributions, one for class 0 and one for class 1, and will predicit an instance
        by the class that outputs the highest likelihood probability for the given instance.
    
        Input
            - ccd0 : An object contating the relevant parameters and methods for the distribution of class 0.
            - ccd1 : An object contating the relevant parameters and methods for the distribution of class 1.
        """
        self.dist0 = ccd0
        self.dist1 = ccd1
    
    def predict(self, x):
        """
        Predicts the instance class using the 2 distribution objects given in the object constructor.
    
        Input
            - An instance to predict.
        Output
            - 0 if the posterior probability of class 0 is higher and 1 otherwise.
        """
        if self.dist0.get_instance_likelihood(x) >= self.dist1.get_instance_likelihood(x):
            return 0
            
        return 1

### Run and evaluate the models

Repeat the process you did for the MAPClassifier, now for the MaxPrior and MaxLikelihood classifiers:
1. Feed the naive_normal distributions and the multi_normal distributions you made for each class into the new models you made in this section
2. Evaluate the accuracies
3. Plot the results as described in the beginning of this section

In [ ]:
###############################
####### YOUR CODE HERE ########
# you may add cells as needed #
###############################

build the classifiers objects containing the 2 distribution objects

In [ ]:
# max prior classifiers
prior_naive_classifier = MaxPrior(naive_normal_CD_0, naive_normal_CD_1)
prior_full_classifier = MaxPrior(multi_normal_CD_0, multi_normal_CD_1)

prior_naive_accuracy = compute_accuracy(test_set, prior_naive_classifier)
prior_full_accuracy = compute_accuracy(test_set, prior_full_classifier)
print(f"For the MaxPrior model, the naive accuracy is {prior_naive_accuracy:.3f} and the full accuracy is {prior_full_accuracy:.3f}.")

In [ ]:
# max likelihood classifiers
likelihood_naive_classifier = MaxLikelihood(naive_normal_CD_0, naive_normal_CD_1)
likelihood_full_classifier = MaxLikelihood(multi_normal_CD_0, multi_normal_CD_1)

likelihood_naive_accuracy = compute_accuracy(test_set, likelihood_naive_classifier)
likelihood_full_accuracy = compute_accuracy(test_set, likelihood_full_classifier)
print(f"For the MaxLikelihood model, the naive accuracy is {likelihood_naive_accuracy:.3f} and the full accuracy is {likelihood_full_accuracy:.3f}.")

Use a plot bar to showcase the models accuracy.

In [ ]:
acc_naive = [prior_naive_accuracy, likelihood_naive_accuracy, naive_accuracy]
acc_full = [prior_full_accuracy, likelihood_full_accuracy, full_accuracy]

x = np.arange(3)

plt.style.use('seaborn')
sns.set(style="darkgrid")
sns.set(font_scale=1.5)

plt.figure(figsize=(14, 10))

plt.bar(x, acc_naive, color="cornflowerblue", width=0.25)
plt.bar(x + 0.25, acc_full, color="mediumaquamarine", width=0.25)

plt.legend(['Naive', 'Full'])
plt.xticks([i + 0.125 for i in range(3)], ['MaxPrior', 'MaxLikelihood', 'MAP'])
plt.title("Models accuracy by model & distribution type".title())
plt.xlabel('Model')
plt.ylabel('Accuracy')

# Discrete Naive Bayes Classifier 

We will now build a discrete naive Bayes based classifier using **Laplace** smoothing.
In the recitation, we saw how to compute the probability for each attribute value under each class:

$$ P(x_j | A_i) = \frac{n_{ij} + 1}{n_i + |V_j|} $$
Where:
* $n_{ij}$ The number of training instances with the class $A_i$ and the value $x_j$ in the relevant attribute.
* $n_i$ The number of training instances with the class $A_i$
* $|V_j|$ The number of possible values of the relevant attribute.

In order to compute the likelihood we assume:
$$ P(x| A_i) = \prod\limits_{j=1}^{n}P(x_j|A_i) $$

And to classify an instance we will choose : 
$$\arg\!\max\limits_{i} P(A_i) \cdot P(x | A_i)$$


## Data
We will try to predict breast cancer again only this time from a different dataset, 
<br> you can read about the dataset here : [Breast Cancer Dataset](https://archive.ics.uci.edu/ml/datasets/breast+cancer)<br>
Load the training set and test set provided for you in the data folder.
 - breast_trainset.csv
 - breast_testset.csv
 

In [ ]:
# Load the train and test set into a pandas dataframe and convert them into a numpy array.
train_set = pd.read_csv('data/breast_trainset.csv').values
test_set = pd.read_csv('data/breast_testset.csv').values

## Build A Discrete Naive Bayes Distribution for each class
Implement the **DiscreteNBClassDistribution** in the next cell and build a distribution object for each class.

In [ ]:
np.where(train_set[: , -1] == 1)

In [ ]:
EPSILLON = 1e-6 # if a certain value only occurs in the test set, the probability for that value will be EPSILLON.

class DiscreteNBClassDistribution():
    def __init__(self, dataset, class_value):
        """
        A class which computes and encapsulate the relevant probabilites for a discrete naive bayes 
        distribution for a specific class. The probabilites are computed with laplace smoothing.
        
        Input
        - dataset: The dataset as a numpy array.
        - class_value: Compute the relevant parameters only for instances from the given class.
        """
        self.class_value_mask = np.where(dataset[: , -1] == class_value)
        self.n_i = len(dataset[self.class_value_mask])
        
        self.dataset = dataset
        self.class_value = class_value

    def get_prior(self):
        """
        Returns the prior porbability of the class according to the dataset distribution.
        """
        return np.sum(self.dataset[: , -1] == self.class_value) / len(self.dataset)
    
    def get_instance_likelihood(self, x):
        """
        Returns the likelihood of the instance under the class according to the dataset distribution.
        """
        xs_likelihoods = list()
        for j, x_j in enumerate(x):
            one_class_dataset = self.dataset[self.class_value_mask]
            n_ij = np.sum(one_class_dataset[: , j] == x_j)  # filter for x_j value feature as well, then count it for n_ij
            v_j = len(np.unique(self.dataset[: , j]))
            likelihood_to_append = (n_ij + 1) / (self.n_i + v_j)

            if n_ij == 0:
                likelihood_to_append = EPSILLON
                
            xs_likelihoods.append( likelihood_to_append )

        # the instance likelihood is the multiplication of the likelihood of all the features (thanks to the NAIVENESS)
        return np.prod(xs_likelihoods)
    
    def get_instance_posterior(self, x):
        """
        Returns the posterior porbability of the instance under the class according to the dataset distribution.
        * Ignoring p(x)
        """
        instance_likelihood = self.get_instance_likelihood(x)
        prior = self.get_prior()
        return instance_likelihood * prior

In [ ]:
discrete_naive_CD_0 = DiscreteNBClassDistribution(train_set, 0)
discrete_naive_CD_1 = DiscreteNBClassDistribution(train_set, 1)

build a MAPClassifier object contating the 2 distribution objects you just made above.

In [ ]:
discrete_naive_classifier = MAPClassifier(discrete_naive_CD_0, discrete_naive_CD_1)

Use the **compute_accuracy** function and the 2 distribution objects you created to compute the accuracy on the test set.

In [ ]:
compute_accuracy(test_set, discrete_naive_classifier)